# **Insper - Eletromagnetismo e Ondulatória** #


**Aula 10 - Modelo elétrico do transformador**

**Item 5** - Calculando correntes e tensões em um transformador AC



In [1]:
#Vamos importar os módulos que precisamos
from cmath import *
from numpy import linalg
import numpy as np
import plotly.graph_objects as go

---
Na célula a seguir:

a) Declare os parâmetros de nosso transformador: $R_1=6\Omega$, $R_2=3\Omega$, $L_1=300mH$, $L_2=30mH$, frequência $f=60 Hz$ e tensão $V_1=50cos(\omega t)$.

b) Calcule:

- Frequência angular $\omega$
- Indutância Mútua $M$
- Reatâncias indutivas de $L_1$ e $L_2$, ou seja $XL1$ e $XL2$
- Reatância indutiva mútua $XM$

Obsrvação: o número complexo $j=\sqrt{-1}$ é nativo da linguagem Python. Para escrever $j$, digite "1j"

In [8]:
frequencias = np.linspace(0, 100e3, 500)
valores_c2 = [150e-9, 0.1e-6, 0.47e-6, 1e-6, 4.7e-6]
Uf = 6    # Tensão de entrada (V1)
Potencia_carga = 3.5    # Potência de carga (P_L)
Rc = pow(Uf, 2) / Potencia_carga    # Resistência de carga (R_C)
R1 = 0.5  # Resistência no primário (R1)
R2 = R1  # Resistência no secundário (R2)
L1 = 0.01 # Indutância do primário (L1)
L2 = L1 # Indutância do secundário (L2)
k = 0.2
XM = k * sqrt(L1 * L2)

In [ ]:
def CalcularTransformadorSerie(Uf, Rc):
    Z=np.array([[R1+L1, -XM],[-XM, L2+R2+Rc]])
    V=np.array([Uf,0])
    i=np.dot(linalg.inv(Z),V)
    return i[0], i[1]

In [10]:
def ajustar_resistencia_pelicular(R_DC, freq):
    return R_DC * (1 + freq / 100e3)

# Função para ajustar a resistência com o efeito pelicular
def ajustar_resistencia_pelicular(R_DC, freq):
    return R_DC * (1 + freq / 100e3)

# Função para simular a tensão V2 e a eficiência com o capacitor C2 em paralelo
def simular_tensao_eficiencia(Uf, Rc, C2, freq):
    omega = 2 * np.pi * freq  # Frequência angular
    XL1 = omega * L1          # Reatância indutiva no primário
    XL2 = omega * L2          # Reatância indutiva no secundário
    XC2 = 1 / (omega * C2)    # Reatância capacitiva no secundário
    
    # Ajustar as resistências pelo efeito pelicular
    R1 = ajustar_resistencia_pelicular(R1_DC, freq)
    R2 = ajustar_resistencia_pelicular(R2_DC, freq)
    
    # Matriz de impedância Z do circuito
    Z = np.array([[R1 + XL1, -XM], 
                  [-XM, XL2 + (R2 * Rc) / (R2 + Rc) - XC2]])
    
    # Vetor de tensões
    V = np.array([Uf, 0])
    i = np.dot(linalg.inv(Z), V)  # Cálculo das correntes
    
    V2 = Uf  # Considerando acoplamento ideal para a tensão no secundário
    
    # Cálculo da eficiência
    I1 = i[0]  # Corrente no primário
    I2 = i[1]  # Corrente no secundário
    P_entrada = np.real(Uf * np.conjugate(I1))  # Potência de entrada
    P_saida = np.real(V2 * np.conjugate(I2))  # Potência de saída
    
    eficiencia = P_saida / P_entrada if P_entrada != 0 else 0
    
    return np.abs(V2), eficiencia

# Inicializando listas para armazenar resultados
resultados_tensao = {}
resultados_eficiencia = {}

# Loop para simular diferentes capacitores C2 e frequências
for C2 in valores_c2:
    V2_list = []  # Lista para armazenar V2 em diferentes frequências
    eficiencia_list = []  # Lista para armazenar eficiência em diferentes frequências
    
    for freq in frequencias:
        V2, eficiencia = simular_tensao_eficiencia(Uf, Rc, C2, freq)
        V2_list.append(V2)  # Armazena a magnitude da tensão V2
        eficiencia_list.append(eficiencia)  # Armazena a eficiência
        
    resultados_tensao[C2] = V2_list
    resultados_eficiencia[C2] = eficiencia_list

# Criar o gráfico usando plotly.graph_objects (plotly.go)
fig = go.Figure()

# Adicionar uma curva para a tensão V2 (linhas sólidas)
for C2, cor in zip(valores_c2, ['blue', 'green', 'red', 'orange', 'purple']):
    fig.add_trace(go.Scatter(
        x=frequencias,
        y=resultados_tensao[C2],
        mode='lines',
        name=f'Tensão V2 (C={C2:.1e} F)',
        line=dict(color=cor, dash='solid')
    ))

# Adicionar uma curva para a eficiência (linhas tracejadas)
for C2, cor in zip(valores_c2, ['blue', 'green', 'red', 'orange', 'purple']):
    fig.add_trace(go.Scatter(
        x=frequencias,
        y=resultados_eficiencia[C2],
        mode='lines',
        name=f'Eficiência (C={C2:.1e} F)',
        line=dict(color=cor, dash='dash')
    ))

# Configurar o layout com dois eixos Y
fig.update_layout(
    title='Tensão V2 e Eficiência por Frequência para Diferentes Valores de Capacitor C2',
    xaxis=dict(title='Frequência (Hz)', type='linear'),
    yaxis=dict(title='Tensão de saída - V2 (V)', side='left', range=[0, 12]),
    yaxis2=dict(title='Eficiência - P2/P1 (%)', overlaying='y', side='right', range=[0, 50]),
    legend=dict(x=1.05, y=1, traceorder='normal'),
)

# Associar as linhas tracejadas ao eixo da eficiência
for i in range(len(valores_c2), len(fig.data)):
    fig.data[i].update(yaxis="y2")

# Mostrar o gráfico
fig.show()

C:\Users\cauep\AppData\Local\Temp\ipykernel_7008\4097599234.py:13: RuntimeWarning:

divide by zero encountered in scalar divide



c) Observe a função que criamos na célula abaixo. Tente explicar o que ela faz.

In [6]:
def CalcularTransformador(Uf, Rc):
    Z=np.array([[R1+XL1, -XM],[-XM, XL2+R2+Rc]])
    V=np.array([Uf,0])
    i=np.dot(linalg.inv(Z),V)
    return i[0], i[1]

d) Calcule as correntes $i_1$ e $i_2$ para três condições de carga ($R_C=1M\Omega$, $R_C=100\Omega$ e $R_C=1\Omega$). Assim que calcular as correntes, calcule também $V_2$ para cada resistor de carga (analise o circuito para saber como calculá-la). Por fim, calcule as relações $N_1/N_2$, $e_1/e_2$ e $V_1/V_2$ e, o mais importante: calcule as potências no primário ($S1$) e no secundário ($S2$).

IMPORTANTE: para dar print nas relações $N_1/N_2$, $V_1/V_2$ e $e_1/e_2$, você precisará usar o **módulo dos fasores**. Para calcular o módulo de um número complexo, use "np.abs(número complexo)".

In [11]:
# Use essa célula para implementar o item d)

Rcs = [1e6, 100, 1]

for Rc in Rcs:
    i1, i2 = CalcularTransformador(Ufonte, Rc)    
    V2 = i2*Rc
    V1 = Ufonte

    e1 = XL1*i1 + XM*i2
    e2 = XM*i1 + XL2*i2
    
    S1 = V1/sqrt(2)*i1.conjugate()/sqrt(2)
    S2 = V2/sqrt(2)*i2.conjugate()/sqrt(2)
    
    print(f'Com o resistor na carga de {Rc} ohms:')
    print(f'i1 (pico): {abs(i1)} A')
    print(f'i2 (pico): {abs(i2)} A')
    print(f'V1 (pico): {abs(V1)} V')
    print(f'V2 (pico): {abs(V2)} V')
    print(f'Relação N1/N2: {abs(sqrt(L1/L2))}')
    print(f'Relação e1/e2: {abs(e1/e2)}')
    print(f'Relação V1/V2: {abs(V1/V2)}')
    print(f'S1 = {S1.real} + {S1.imag}j VA')
    print(f'S2 = {S2.real} + {S2.imag}j VA')
    print(f'Eficiência: {S2.real/S1.imag}')
    print('\n')

Com o resistor na carga de 1000000.0 ohms:
i1 (pico): 0.044209058049595525 A
i2 (pico): 1.581110888184443e-05 A
V1 (pico): 50 V
V2 (pico): 15.811108881844428 V
Relação N1/N2: 3.1622776601683795
Relação e1/e2: 3.1622776601683795
Relação V1/V2: 3.16233354495547
S1 = 0.0059883183979210376 + 1.105210228220442j VA
S2 = 0.0001249955820367699 + 0.0j VA
Eficiência: 0.00011309665694827305


Com o resistor na carga de 100 ohms:
i1 (pico): 0.06527694651469498 A
i2 (pico): 0.15261745532108567 A
V1 (pico): 50 V
V2 (pico): 15.261745532108566 V
Relação N1/N2: 3.1622776601683795
Relação e1/e2: 3.162277660168379
Relação V1/V2: 3.2761652259767424
S1 = 1.2123257541760513 + 1.092447301789855j VA
S2 = 1.1646043834341788 + 0.0j VA
Eficiência: 1.0660508580378223


Com o resistor na carga de 1 ohms:
i1 (pico): 1.0876245616424591 A
i2 (pico): 3.4372217518608763 A
V1 (pico): 50 V
V2 (pico): 3.4372217518608763 V
Relação N1/N2: 3.1622776601683795
Relação e1/e2: 3.1622776601683795
Relação V1/V2: 14.546632021320857

e) Recalcule o item **d)**, mas agora considerando que o transformador é **ideal**. Para isso, basta zerar as resistências R1 e R2 do primário e do secundário. Após zer-a-las, copie o código do item d) na célula abaixo. O que você pode concluir sobre a potência real nas três condições de carga?

In [5]:
# Use essa célula para implementar o item e)



